In [ ]:
import h5py
import numpy as np
file_path = '/Users/jsmallwood/Documents/projects/cuda-spatial-filtering/scripts/cap_251127_1204.hdf5'
with h5py.File(file_path, "r") as hdf:

    # List all groups in the file
    print("Keys in the file:")
    for key in hdf.keys():
        print(" -", key)
    dataset_name = "visibilities"
    if dataset_name in hdf:
        data = hdf[dataset_name][:]
        print(f"\nData from '{dataset_name}':")
        #print(data)
    else:
        print(f"\nDataset '{dataset_name}' not found in the file.")

In [ ]:
data.shape

In [ ]:
data[-1]

In [ ]:

def unpack_triangular_corr(data):
    """
    data shape: (T, C, B, P1, P2, 2)
    where B = N(N+1)/2
    returns: (T, C, N, N, P1, P2)
    """
    T, C, B, P1, P2, _ = data.shape

    # infer N
    N = int((np.sqrt(8*B + 1) - 1) // 2)
    assert N*(N+1)//2 == B

    # convert real/imag → complex
    d = data[...,0] + 1j*data[...,1]

    # correct (a, b) mapping with no overflow
    k = np.arange(B)
    Tn = np.arange(N+1)
    Tn = Tn*(Tn+1)//2                       # triangular sequence

    a = np.searchsorted(Tn, k+1) -1           # ensures a < N+1 always
    start = Tn[a]
    b = k - start
    # output: full Hermitian matrix
    R = np.zeros((T, C, N, N, P1, P2), dtype=d.dtype)

    # fill lower triangle
    R[:, :, a, b, :, :] = d

    # fill upper using Hermitian symmetry
    R[:, :, b, a, :, :] = np.conj(d)

    return R

In [ ]:
pol = 0
channel = 4
t_samp = 4
d = unpack_triangular_corr(data)
d[t_samp, channel, 0:10, 0:10, pol,pol] 


In [ ]:
pol = 0
channel = 7
t_samp = 0
d = unpack_triangular_corr(data)
print(d[t_samp, channel, :2, :2, 0,1])
print(d[t_samp, channel, :2, :2, 1,0])

In [ ]:
mag = np.abs(d)

import matplotlib.pyplot as plt

x = np.arange(8)

plt.figure(figsize=(12, 8))

for idx in range(0,10):              
    for a in range(0,10):              
        if idx != a and a < idx:
            continue
        for b in range(2):           
            for c in range(2):       
                y = np.log10(mag[0, :, idx, a, b, c])

                if np.any(y != 0):   # skip if it's entirely zero
                    plt.plot(x, y, label=f"R{idx}-R{a}-P{b}-P{c}")

plt.legend()
plt.xlabel("Channel")
plt.ylabel("Log10 Magnitude")
plt.title("Magnitude")
plt.tight_layout()
plt.show()

In [ ]:
mag = np.abs(d)

import matplotlib.pyplot as plt

x = np.arange(8)

plt.figure(figsize=(12, 8))

for idx in range(0,10):              
    for a in range(0,10):              
        if idx != a and a < idx:
            continue
        for b in [0]:           
            for c in [0]:       
                y = np.angle(d[0, :, idx, a, b, c])

                if np.any(y != 0):   # skip if it's entirely zero
                    plt.plot(x, y, label=f"R{idx}-R{a}-P{b}-P{c}")

plt.legend()
plt.xlabel("Channel")
plt.ylabel("Phase")
plt.title("Phase")
plt.tight_layout()
plt.show()

In [ ]:
pol = 0
channel = 1
t_samp = 0
evals,evecs = np.linalg.eigh(d[t_samp, channel, :, :, pol,pol])

In [ ]:
evals

In [ ]:
evecs[:,-1]

In [ ]:
channel_evecs= []
for channel in range(8):
    evals,evecs = np.linalg.eigh(d[t_samp, channel, :, :, pol,pol])
    dom = evecs[:,-1]
    channel_evecs.append(dom)
    

In [ ]:
np.abs(channel_evecs[0].conj().T @ channel_evecs[7])

In [ ]:
64000000 / 4 / 8

In [ ]:
800000000 / 1024

In [ ]:
1000000000/1024